In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import json
%matplotlib inline


In [4]:
with open("data1.json",'rb') as datafile:
  song = json.load(datafile)
song = pd.DataFrame(song)

In [5]:
song

,endTime,artistName,trackName,msPlayed
0,2020-04-01 19:34,The Maine,Lovely,18808
1,2020-04-01 21:35,Conan Gray,Maniac,50928
2,2020-04-02 01:31,The Wrecks,Out Of Style,160080
3,2020-04-02 01:50,THE DRIVER ERA,Preacher Man,219389
4,2020-04-02 01:51,Fitz and The Tantrums,I Just Wanna Shine,4942
...,...,...,...,...
9995,2020-12-11 20:02,John-Robert,Pelican,157299
9996,2020-12-11 20:05,Sum 41,Fat Lip,178266
9997,2020-12-11 20:08,Panama Wedding,Uma,179025
9998,2020-12-11 20:12,The Happy Fits,Dirty Imbecile,239612


In [8]:
pip install --user spotipy

  Using cached spotipy-2.17.1-py3-none-any.whl (26 kB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached urllib3-1.26.4-py2.py3-none-any.whl (153 kB)
Note: you may need to restart the kernel to use updated packages.


In [16]:
## access the spotify api 
import spotipy.util as util

username = '22ncyzdcn5gqnknz3ska32qxq'
client_id ='c8f0e81eb3e247d49dd3f503cc04930d'
client_secret = '1d50b1f2515f4c249c9453d267b0564f'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [17]:
print(token)

BQBfYpuG4hWCzRTIvdimQMjRf-YXFxkZ4LOLNY_A08KHDEEqq7kE6hbMGG6lRk5D_trY0UnnEEqKc4P7BlF84XHkQyky6bxpc_QsaX9WCktJk0U_HpW8Y3D_7iZ_5YNbyCqtX6B-0mgxfnkAFb8Hcj6O9CnaOUfgoxddNsY


In [7]:
import requests
import spotipy.util as util
from spotipy import Spotify

In [8]:
def get_api_id(track_info: str, token: str, artist: str = None) -> str:
    
    '''Performs a query on Spotify API to get a track ID.
    See https://curl.trillworks.com/'''

    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,
    }
    track_name = track_info.split("___")[0]
    params = [
    ('q', track_name),
    ('type', 'track'),
    ]
    artist = track_info.split("___")[-1]
    if artist:
        params.append(('artist', artist))
        
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        results = json['tracks']['items']
        first_result = json['tracks']['items'][0]
        # Check if searched artist is in response as the first one isn't
        # necessarily the right one
        if artist:
            for result in results:
                if artist.strip() == result['artists'][0]['name'].strip():
                    track_id = result['id']
                    return track_id
        # If specific artist is not found from results, use the first one
        track_id = first_result['id']
        return track_id
    except:
        return None

In [13]:
lucy_id = get_api_id('Lovely', token, artist = 'The Maine')
print(lucy_id)

0u2P5u6lvoDfwTYjAADbn4


In [22]:
id = []
for i in range(song.shape[0]):
    if i < 5:
        id.append(get_api_id(song.iloc[i,2],token, artist=song.iloc[i,1]))
    else:
        break
    

['0u2P5u6lvoDfwTYjAADbn4', '6KfoDhO4XUWSbnyKjNp9c4', '62ewcS8yoxhFU59yhVlL9r', '0oeW7qeQnBULDmDWsq4w5o', '4GZ3YCkuH0VvTluVLwUp4g']


In [9]:
def get_features(track_id: str, token: str) -> dict:
    sp = Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [17]:
lucy_features = get_features(lucy_id, token)
lucy_features

{'danceability': 0.351,
 'energy': 0.296,
 'key': 4,
 'loudness': -10.109,
 'mode': 0,
 'speechiness': 0.0333,
 'acousticness': 0.934,
 'instrumentalness': 0,
 'liveness': 0.095,
 'valence': 0.12,
 'tempo': 115.284,
 'type': 'audio_features',
 'id': '0u2P5u6lvoDfwTYjAADbn4',
 'uri': 'spotify:track:0u2P5u6lvoDfwTYjAADbn4',
 'track_href': 'https://api.spotify.com/v1/tracks/0u2P5u6lvoDfwTYjAADbn4',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0u2P5u6lvoDfwTYjAADbn4',
 'duration_ms': 200186,
 'time_signature': 4}

In [71]:
column_names = list(lucy_features.keys())[:11]
features = pd.DataFrame(columns = column_names)

In [72]:
features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence


In [67]:
def get_feature_history(df:dataframe):
    for i in df:
        id = get_api_id(i, token, artist = 'The Beatles')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
1,0.647,0.537,6,-12.006,0,0.21,0.43,0.000258,0.119,0.846,84.46,audio_features,5RxpYHVbGJPOvSEATQyg9P,spotify:track:5RxpYHVbGJPOvSEATQyg9P,https://api.spotify.com/v1/tracks/5RxpYHVbGJPO...,https://api.spotify.com/v1/audio-analysis/5Rxp...,142500,4


In [10]:
def id_df(df):
    return get_api_id(df[2], token, df[1])

In [11]:
song['id'] = song.apply(id_df, axis=1)

In [23]:
song.head()

,endTime,artistName,trackName,msPlayed,id,features
0,2020-04-01 19:34,The Maine,Lovely,18808,0u2P5u6lvoDfwTYjAADbn4,None
1,2020-04-01 21:35,Conan Gray,Maniac,50928,6KfoDhO4XUWSbnyKjNp9c4,None
2,2020-04-02 01:31,The Wrecks,Out Of Style,160080,62ewcS8yoxhFU59yhVlL9r,None
3,2020-04-02 01:50,THE DRIVER ERA,Preacher Man,219389,0oeW7qeQnBULDmDWsq4w5o,None
4,2020-04-02 01:51,Fitz and The Tantrums,I Just Wanna Shine,4942,4GZ3YCkuH0VvTluVLwUp4g,None


In [12]:
song.to_csv('song_id.csv',sep='\t')

In [13]:
song['features'] = get_features(song['id'],token)
song.to_csv('song_feature.csv',sep='\t')

In [29]:
features = []
for i in range(song.shape[0]):
        features.append(get_features(song.iloc[i,4],token))


In [41]:
song = song.drop(['features'],axis=1)

In [40]:
print(len(features))
print(song.shape)

10000
(10000, 6)


In [42]:
sum(song['id'].isnull())

24

In [43]:
song[pd.isnull(song).any(axis=1)]

,endTime,artistName,trackName,msPlayed,id
2060,2020-05-09 07:25,music that isn’t already on here 2.0,Kaleidoscope Eyes - Panic! at the Disco,201621,None
2062,2020-05-09 07:30,music that isn’t already on here 2.0,Turn Off The Lights - Panic! at the Disco,239076,None
2063,2020-05-09 07:34,music that isn’t already on here 2.0,Bittersweet - Panic! at the Disco,215052,None
2152,2020-05-10 04:18,music that isn’t already on here 2.0,Turn Off The Lights - Panic! at the Disco,72,None
2153,2020-05-10 04:18,music that isn’t already on here 2.0,Turn Off The Lights - Panic! at the Disco,3040,None
2155,2020-05-10 04:18,music that isn’t already on here 2.0,Kaleidoscope Eyes - Panic! at the Disco,22448,None
2156,2020-05-10 04:35,music that isn’t already on here 2.0,Kaleidoscope Eyes - Panic! at the Disco,192810,None
2157,2020-05-10 04:41,music that isn’t already on here 2.0,Turn Off The Lights - Panic! at the Disco,40160,None
2204,2020-05-12 17:04,music that isn’t already on here 2.0,Modern Day Cain (slow jam version) - IDKHBTFM,157111,None
2205,2020-05-12 17:07,music that isn’t already on here 2.0,Mr Sinister - IDKHBTFM,100671,None


In [47]:
null_list = song[pd.isnull(song).any(axis=1)].index.tolist()

In [50]:
print(null_list)
print(features[2060])

[2060, 2062, 2063, 2152, 2153, 2155, 2156, 2157, 2204, 2205, 2206, 2207, 4997, 5678, 7163, 7164, 7165, 7166, 7167, 7214, 7479, 7480, 7849, 8464]
None


In [51]:
features_nonull = features
features_nonull = filter(None,features_nonull)

In [52]:
df = pd.DataFrame(features_nonull)

In [53]:
df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.351,0.296,4,-10.109,0,0.0333,0.93400,0.000000,0.0950,0.120,115.284,audio_features,0u2P5u6lvoDfwTYjAADbn4,spotify:track:0u2P5u6lvoDfwTYjAADbn4,https://api.spotify.com/v1/tracks/0u2P5u6lvoDf...,https://api.spotify.com/v1/audio-analysis/0u2P...,200186,4
1,0.628,0.639,8,-5.460,1,0.0435,0.00162,0.000000,0.3540,0.493,108.045,audio_features,6KfoDhO4XUWSbnyKjNp9c4,spotify:track:6KfoDhO4XUWSbnyKjNp9c4,https://api.spotify.com/v1/tracks/6KfoDhO4XUWS...,https://api.spotify.com/v1/audio-analysis/6Kfo...,185773,4
2,0.595,0.848,9,-5.481,0,0.0602,0.00399,0.000000,0.3380,0.801,140.031,audio_features,62ewcS8yoxhFU59yhVlL9r,spotify:track:62ewcS8yoxhFU59yhVlL9r,https://api.spotify.com/v1/tracks/62ewcS8yoxhF...,https://api.spotify.com/v1/audio-analysis/62ew...,160080,4
3,0.689,0.947,1,-3.926,1,0.0897,0.00209,0.000000,0.2280,0.393,119.993,audio_features,0oeW7qeQnBULDmDWsq4w5o,spotify:track:0oeW7qeQnBULDmDWsq4w5o,https://api.spotify.com/v1/tracks/0oeW7qeQnBUL...,https://api.spotify.com/v1/audio-analysis/0oeW...,219390,4
4,0.705,0.918,6,-2.770,0,0.0684,0.04000,0.000093,0.0803,0.546,92.511,audio_features,4GZ3YCkuH0VvTluVLwUp4g,spotify:track:4GZ3YCkuH0VvTluVLwUp4g,https://api.spotify.com/v1/tracks/4GZ3YCkuH0Vv...,https://api.spotify.com/v1/audio-analysis/4GZ3...,207925,4


In [83]:
sum(pd.isnull(dataframe['id']))

497

In [54]:
final_1 = pd.merge(song, df, how = 'left', on=['id'])
final_1 = final_1.drop_duplicates()
final_1 = final_1.dropna()

In [56]:
final_1.to_csv(r'final_1.txt',sep='\t',index=None)